### TEST 1

In [7]:
import pandas as pd

# --- helper (unchanged) -------------------------------------------------
def _rounds_from_score(score_str: str) -> int | None:
    if not isinstance(score_str, str):
        return None
    try:
        a, b = score_str.replace("-", "-").split("-")   # handles weird dash
        return int(a) + int(b)
    except ValueError:
        return None

DEFAULT_AGENT_ROLE = {  # ← same mapping as before
    # Duelists
    "raze": "Duelist", "jett": "Duelist", "reyna": "Duelist",
    "yoru": "Duelist", "phoenix": "Duelist", "neon": "Duelist",
    # Initiators
    "sova": "Initiator", "skye": "Initiator", "fade": "Initiator",
    "gekko": "Initiator", "breach": "Initiator", "kayo": "Initiator",
    # Controllers
    "omen": "Controller", "brimstone": "Controller", "astra": "Controller",
    "harbor": "Controller", "vyse": "Controller",
    # Sentinels
    "cypher": "Sentinel", "killjoy": "Sentinel", "sage": "Sentinel",
    "deadlock": "Sentinel", "chamber": "Sentinel",
}

# --- main builder (dictionary style .agg) -------------------------------
def build_player_kill_dataset(
    csv_path: str,
    agent_role_map: dict[str, str] | None = None,
    return_full: bool = False,
):
    role_map = (agent_role_map or DEFAULT_AGENT_ROLE).copy()

    df = pd.read_csv(csv_path)
    df["RoundsPlayed"] = df["MapScore"].apply(_rounds_from_score)

    agg = (df
           .groupby(["MatchID", "Player", "Agent"])
           .agg({
               "Kills": "sum",
               "ACS": "mean",
               "Assists": "sum",
               "Deaths": "sum",
               "FirstKills": "sum",
               "RoundsPlayed": "sum"
           })
           .reset_index())

    # map agent → role
    agg["Agent_lc"] = agg["Agent"].str.lower()
    agg["Role"] = agg["Agent_lc"].map(role_map).fillna("Unknown")

    # one-hot encode role
    encoded = pd.get_dummies(agg, columns=["Role"], prefix="Role")

    # split features / target
    X = encoded.drop(columns=["MatchID", "Player", "Agent",
                              "Agent_lc", "Kills"])
    y = encoded["Kills"]

    return (X, y, encoded) if return_full else (X, y)


In [8]:
# EMEA example
X, y = build_player_kill_dataset("emea.csv")

# If you want to inspect
X, y, df_dbg = build_player_kill_dataset("emea.csv", return_full=True)
print(X.head())


     ACS  Assists  Deaths  FirstKills RoundsPlayed  Role_Controller  \
0  147.0        3      14         0.0            0            False   
1  215.0        5      13         2.0            0            False   
2  304.5       18      20         3.0            0             True   
3  233.5       10      27        13.0            0            False   
4  159.5       13      20         2.0            0            False   

   Role_Duelist  Role_Initiator  Role_Sentinel  Role_Unknown  
0         False            True          False         False  
1         False           False          False          True  
2         False           False          False         False  
3          True           False          False         False  
4         False            True          False         False  


In [49]:
from xgboost import XGBRegressor
import numpy as np

n_boot = 500
preds = np.empty((n_boot, len(y)))

for i in range(n_boot):
    sample_idx = np.random.choice(len(y), replace=True, size=len(y))
    X_s, y_s = X.iloc[sample_idx], y.iloc[sample_idx]
    model = XGBRegressor(max_depth=6, n_estimators=200, learning_rate=0.1)
    model.fit(X_s, y_s)
    preds[i] = model.predict(X)

# mean prediction & 95 % CI for each row
pred_mean = preds.mean(axis=0)
pred_low  = np.percentile(preds, 2.5, axis=0)
pred_high = np.percentile(preds, 97.5, axis=0)


KeyboardInterrupt: 

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

# ------------------------------------------------------------
# 1.  Build dataset
# ------------------------------------------------------------
def build_player_map_opponent_dataset(csv_path: str):
    """Return X (one-hot), y, groups (MatchID), and raw df."""
    df = pd.read_csv(
        csv_path,
        usecols=["MatchID", "Map", "Team", "Player", "Kills"]
    )

    # ---------- derive OpponentTeam ----------------------------------------
    def get_opponent(series):
        teams = series.unique()
        if len(teams) == 2:
            return series.replace({teams[0]: teams[1], teams[1]: teams[0]})
        return series  # fallback; shouldn’t happen
    df["OpponentTeam"] = (
        df.groupby(["MatchID", "Map"])["Team"]
          .transform(get_opponent)
    )

    # ---------- one-hot encode three inputs --------------------------------
    X = pd.get_dummies(
            df[["Player", "Map", "OpponentTeam"]],
            drop_first=False
        )
    y = df["Kills"]
    groups = df["MatchID"]        # keep matches intact in split
    return X, y, groups, df

# ------------------------------------------------------------
# 2.  Load any region file
#     (swap path to "amer.csv" / "apac.csv" etc.)
# ------------------------------------------------------------
X, y, groups, raw = build_player_map_opponent_dataset("emea.csv")

# ------------------------------------------------------------
# 3.  Train / test split – group-aware (80 / 20)
# ------------------------------------------------------------
gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
tr_idx, te_idx = next(gss.split(X, y, groups))
X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]

# ------------------------------------------------------------
# 4.  Fit XGBoost
# ------------------------------------------------------------
model = XGBRegressor(
    max_depth=6,
    n_estimators=400,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="reg:squarederror",
    random_state=0,
    n_jobs=4
)
model.fit(X_tr, y_tr)

# ------------------------------------------------------------
# 5.  Evaluate
# ------------------------------------------------------------
pred = model.predict(X_te)
print(f"MAE : {mean_absolute_error(y_te, pred):.2f}")
print(f"R²  : {r2_score(y_te, pred):.3f}")

# ------------------------------------------------------------
# 6.  Inspect a few predictions
# ------------------------------------------------------------
preview = (raw.iloc[te_idx]
           .loc[:, ["MatchID", "Player", "Team", "OpponentTeam", "Map", "Kills"]]
           .assign(Pred=np.round(pred, 2))
           .head(15))
print("\nSample predictions:")
print(preview.to_string(index=False))


MAE : 5.84
R²  : -0.179

Sample predictions:
 MatchID    Player Team OpponentTeam      Map  Kills      Pred
       9   MrFaliN  FUT           M8   Icebox     17 15.700000
       9      xeus  FUT           M8   Icebox     19 20.360001
       9     qRaxs  FUT           M8   Icebox     13 12.430000
       9 AtaKaptan  FUT           M8   Icebox     13 13.970000
       9   yetujey  FUT           M8   Icebox     11 14.000000
       9     kamyk   M8          FUT   Icebox     22 12.330000
       9   K4DAVRA   M8          FUT   Icebox     17 13.240000
       9     Minny   M8          FUT   Icebox     16 14.990000
       9    Zyppan   M8          FUT   Icebox     11  8.860000
       9     Click   M8          FUT   Icebox     19 11.010000
       9 AtaKaptan  FUT           M8 Fracture     23 15.180000
       9      xeus  FUT           M8 Fracture     29 21.340000
       9   MrFaliN  FUT           M8 Fracture     28 16.830000
       9   yetujey  FUT           M8 Fracture     26 15.210000
       9  

In [17]:
import joblib

# after `model.fit(X_tr, y_tr)` is done
joblib.dump(model, "xgb_player_map_opponent.pkl")
X.columns.to_series().to_pickle("model_columns.pkl")   # exact column order


In [18]:
import pandas as pd
import numpy as np
import joblib

def predict_kills(player, map_name, opp_team,
                  model_path="xgb_player_map_opponent.pkl",
                  cols_path="model_columns.pkl"):
    # --- load model & column index ----------------------------
    model = joblib.load(model_path)
    cols  = pd.read_pickle(cols_path)          # Index of training columns

    # --- create 1-row DataFrame with the three inputs ----------
    row = pd.DataFrame(
        {"Player":[player], "Map":[map_name], "OpponentTeam":[opp_team]}
    )

    # one-hot encode using the *same* logic as training
    row_enc = pd.get_dummies(row, drop_first=False)

    # add any missing columns, keep correct order
    row_enc = row_enc.reindex(columns=cols, fill_value=0)

    # --- predict ----------------------------------------------
    pred = model.predict(row_enc)[0]
    return pred


In [20]:
predict_kills("nAts", "Icebox", "NAVI")


16.241686

### TEST 2

In [74]:
import pandas as pd, numpy as np, joblib
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

# ── helper ──────────────────────────────────────────────────────────────
def rounds_from_score(s: str) -> float:
    try:
        a, b = s.replace("-", "-").split("-")   # handle weird dash
        return int(a) + int(b)
    except Exception:
        return np.nan                           # filled later
def build_xy(csv_path: str):
    keep = ["MatchID", "Map", "Team", "Player", "Kills", "MapScore"]
    df   = pd.read_csv(csv_path, usecols=keep)

    # 1. numeric map length
    df["RoundsPlayed"] = df["MapScore"].str.replace("-", "-").str.split("-").apply(
        lambda ab: pd.to_numeric(ab).sum() if isinstance(ab, list) else np.nan
    )
    df["RoundsPlayed"].fillna(df["RoundsPlayed"].median(), inplace=True)

    # 2. create OpponentTeam  (⇄ swap the two team strings)
    def swap(series):
        uniq = series.unique()
        if len(uniq) == 2:
            return series.replace({uniq[0]: uniq[1], uniq[1]: uniq[0]})
        return np.nan                          # safeguard
    df["OpponentTeam"] = df.groupby(["MatchID", "Map"])["Team"].transform(swap)

    # make sure we didn't accidentally introduce NaNs
    assert df["OpponentTeam"].isna().sum() == 0, "OpponentTeam has NaNs!"

    # 3. one-hot encode Player, Map, OpponentTeam
    Xcat = pd.get_dummies(df[["Player", "Map", "OpponentTeam"]], drop_first=False)
    X    = pd.concat([Xcat, df[["RoundsPlayed"]].astype(float)], axis=1)
    y    = df["Kills"]
    groups = df["MatchID"]
    return X, y, groups, df


In [78]:
import joblib
import pandas as pd

# 1) save the XGBoost model
joblib.dump(model, "kills_model.pkl")

# 2) save the exact training column order (X is your design-matrix DataFrame)
pd.Series(X.columns).to_pickle("kills_model_cols.pkl")

print("✅  kills_model.pkl and kills_model_cols.pkl written to disk.")


✅  kills_model.pkl and kills_model_cols.pkl written to disk.


In [79]:
def predict_kills(player, map_name, opponent_team, rounds_played,
                  model_path="kills_model.pkl",
                  cols_path="kills_model_cols.pkl") -> float:
    """Return predicted kills for the given inputs."""
    model = joblib.load(model_path)
    cols  = pd.read_pickle(cols_path)     # training column order

    row = pd.DataFrame({
        "Player":       [player],
        "Map":          [map_name],
        "OpponentTeam": [opponent_team],
        "RoundsPlayed": [float(rounds_played)]
    })

    row_enc = pd.get_dummies(row, drop_first=False)
    row_enc = row_enc.reindex(columns=cols, fill_value=0)   # add missing dummies

    return model.predict(row_enc)[0]

# ── demo ───────────────────────────────────────────────────────────────
est = predict_kills("kamo", "Split", "FNC", 24)
print(f"Predicted kills: {est:.1f}")


Predicted kills: 15.3


In [80]:
import pandas as pd
print(sorted(pd.read_pickle("kills_model_cols.pkl")
             .str.replace("OpponentTeam_", "")
             .loc[lambda s: s.str.startswith("OpponentTeam_")]))


[]


In [73]:
import pandas as pd

df  = pd.read_csv("emea.csv", usecols=["MatchID", "Map", "Team"])
# same transformation we used in make_xy()
df["OpponentTeam"] = (
    df.groupby(["MatchID", "Map"])["Team"]
      .transform(lambda s: s.replace(dict(zip(s.unique(), s.unique()[::-1]))) )
)

print(df[["Team", "OpponentTeam"]].head(10))
print(df["OpponentTeam"].isna().mean(), "fraction NaNs")


  Team OpponentTeam
0  VIT           GX
1  VIT           GX
2  VIT           GX
3  VIT           GX
4  VIT           GX
5   GX          VIT
6   GX          VIT
7   GX          VIT
8   GX          VIT
9   GX          VIT
0.0 fraction NaNs


### TEST 3

In [67]:
emea_df = pd.read_csv("emea.csv")
amer_df = pd.read_csv("amer.csv")
cn_df   = pd.read_csv("cn.csv")
apac_df = pd.read_csv("apac.csv")

emea_df["Region"] = "EMEA"
amer_df["Region"] = "AMER"
cn_df["Region"]   = "CN"
apac_df["Region"] = "APAC"

# combine all regions into one DataFrame
all_df = pd.concat([emea_df, amer_df, cn_df, apac_df], ignore_index=True)
all_df["MatchID"] = all_df["MatchID"].astype(str)
all_df["Match_ID"] = all_df["MatchID"] + "_" + all_df["Region"]
all_df.to_csv("all_regions.csv", index=False)


In [79]:
"""
Train an XGBoost model that predicts a player's kills on a map
from   Player  +  Map  +  OpponentTeam  +  RoundsPlayed.
Saves:
    kills_model.pkl        – trained XGBRegressor
    kills_model_cols.pkl   – ordered Index of one-hot feature columns
"""

import pandas as pd, numpy as np, joblib, os
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

# -----------------------------------------------------------------------
# 0.  CONFIGURATION  (adjust if you like)
# -----------------------------------------------------------------------

CSV_PATH   = "all_regions.csv"       # <-- change to amer.csv, apac.csv ...
TEST_SIZE  = 0.20                   # 20 % of matches for evaluation
RANDOM_SEED = 42
MODEL_OUT  = "kills_model.pkl"
COLS_OUT   = "kills_model_cols.pkl"

# -----------------------------------------------------------------------
# 1.  Helper: "6-13"  ->  19 rounds
# -----------------------------------------------------------------------
def rounds_from_score(s: str) -> float:
    try:
        a, b = s.replace("-", "-").split("-")   # handles weird dash char
        return int(a) + int(b)
    except Exception:
        return np.nan

# -----------------------------------------------------------------------
# 2.  Build design matrix X, target y, and grouping key
# -----------------------------------------------------------------------
def build_xy(csv_path: str):
    usecols = ["MatchID", "Map", "Team", "Player", "Kills", "MapScore"]
    df = pd.read_csv(csv_path, usecols=usecols)

    # 2.1 numeric match length
    df["RoundsPlayed"] = df["MapScore"].apply(rounds_from_score)
    df["RoundsPlayed"].fillna(df["RoundsPlayed"].median(), inplace=True)

    # 2.2 derive OpponentTeam by swapping the two teams on each (Match,Map)
    def swap(series):
        t1, t2 = series.unique()[:2]           # exactly two unique teams
        return series.replace({t1: t2, t2: t1})
    df["OpponentTeam"] = (
        df.groupby(["MatchID", "Map"])["Team"]
          .transform(swap)
    )

    # 2.3 one-hot encode the 3 categoricals; keep numeric RoundsPlayed
    X_cat = pd.get_dummies(df[["Player", "Map", "OpponentTeam"]],
                           drop_first=False)
    X = pd.concat([X_cat, df[["RoundsPlayed"]].astype(float)], axis=1)
    y = df["Kills"]
    groups = df["MatchID"]                     # keep matches intact in split
    return X, y, groups

X, y, groups = build_xy(CSV_PATH)

# -----------------------------------------------------------------------
# 3.  Group-aware train/test split  (no leakage between same match)
# -----------------------------------------------------------------------
gss = GroupShuffleSplit(test_size=TEST_SIZE,
                        n_splits=1,
                        random_state=RANDOM_SEED)
tr_idx, te_idx = next(gss.split(X, y, groups))
X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]

# -----------------------------------------------------------------------
# 4.  Fit XGBoost regressor
# -----------------------------------------------------------------------
model = XGBRegressor(
    max_depth=5,
    n_estimators=400,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="reg:squarederror",
    n_jobs=4,
    random_state=RANDOM_SEED
).fit(X_tr, y_tr)

# -----------------------------------------------------------------------
# 5.  Evaluate
# -----------------------------------------------------------------------
print(f"Test MAE : {mean_absolute_error(y_te, model.predict(X_te)):.2f}")
print(f"Test R²  : {r2_score(y_te,      model.predict(X_te)):.3f}")

# -----------------------------------------------------------------------
# 6.  Save artefacts
# -----------------------------------------------------------------------
joblib.dump(model, MODEL_OUT)
pd.Series(X.columns).to_pickle(COLS_OUT)
print(f"✅  Saved: {MODEL_OUT}, {COLS_OUT}  ({len(X.columns)} features)")


/var/folders/3g/fzqltvr90_11xv3v1sxb4f2m0000gn/T/ipykernel_94765/1975378772.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["RoundsPlayed"].fillna(df["RoundsPlayed"].median(), inplace=True)


Test MAE : 4.14
Test R²  : 0.018
✅  Saved: kills_model.pkl, kills_model_cols.pkl  (310 features)


In [76]:


MODEL_PATH = "kills_model.pkl"
COLS_PATH  = "kills_model_cols.pkl"

def predict_kills(player: str,
                  map_name: str,
                  opponent_team: str,
                  rounds_played: int | float,
                  model_path: str = MODEL_PATH,
                  cols_path: str = COLS_PATH) -> float:
    """Return estimated kills (float)."""
    # 1. load model + training column order
    model = joblib.load(model_path)
    cols  = pd.read_pickle(cols_path)   # pandas Index

    # 2. build 1-row DataFrame with the four inputs
    row = pd.DataFrame({
        "Player":       [player],
        "Map":          [map_name],
        "OpponentTeam": [opponent_team],
        "RoundsPlayed": [float(rounds_played)]
    })

    # 3. one-hot encode & align columns
    row_enc = pd.get_dummies(row, drop_first=False)
    row_enc = row_enc.reindex(columns=cols, fill_value=0)

    # 4. predict
    return model.predict(row_enc)[0]

# # ── demo ────────────────────────────────────────────────────────────────
# if __name__ == "__main__":
#     est = predict_kills("paTiTek", "Split", "FNC", 23)
#     print(f"Predicted kills: {est:.1f}")


In [77]:
import pandas as pd, numpy as np, re, joblib
from pathlib import Path

def score_kills_csv(csv_path,
                    model_path="kills_model.pkl",
                    cols_path="kills_model_cols.pkl",
                    out_path=None):

    df = pd.read_csv(csv_path)

    # ---- 0) canonicalise the four must-have columns ------------------
    ren = {
        "Match": "MatchID",
        r"map(|name)|map_title":               "Map",
        r"team$|my[_\s]?team|squad":           "Team",
        r"player(|name)|gamer(tag)?|nick":     "Player",
    }
    for pat, new in ren.items():
        for c in df.columns:
            if re.fullmatch(pat, c, flags=re.I):
                df = df.rename(columns={c: new})

    required = {"MatchID", "Map", "Team", "Player"}
    if not required.issubset(df.columns):
        missing = required - set(df.columns)
        raise ValueError(f"CSV still missing columns: {missing}")

    # ---- 1) RoundsPlayed (parse if absent) ---------------------------
    if "RoundsPlayed" not in df.columns:
        if "MapScore" not in df.columns:
            raise ValueError("Need RoundsPlayed or MapScore column.")
        df["RoundsPlayed"] = (
            df["MapScore"].astype(str)
              .apply(lambda s: sum(map(int, re.findall(r"\d+", s)[:2]))
                                  if re.search(r"\d+.*\d+", s) else np.nan)
        )
    df["RoundsPlayed"] = pd.to_numeric(df["RoundsPlayed"], errors="coerce")

    # ---- 2) OpponentTeam by swapping ---------------------------------
    def swap(s):
        u = s.unique()
        if len(u) != 2:
            raise ValueError("Each MatchID–Map must have exactly 2 teams")
        return s.replace({u[0]: u[1], u[1]: u[0]})
    df["OpponentTeam"] = df.groupby(["MatchID", "Map"])["Team"].transform(swap)

    # ---- 3) build feature matrix & score -----------------------------
    X = pd.get_dummies(df[["Player", "Map", "OpponentTeam", "RoundsPlayed"]],
                       drop_first=False)
    X = X.reindex(columns=pd.read_pickle(cols_path), fill_value=0)

    df["PredKills"] = joblib.load(model_path).predict(X)

    if out_path:
        Path(out_path).parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(out_path, index=False)

    return df


In [74]:
scored = score_kills_csv("test.csv", out_path="test_scored.csv")
scored


,Map,Team,Player,Agent,ACS,Kills,Deaths,Assists,FirstKills,Winner,MapScore,Date,MatchID,RoundsPlayed,OpponentTeam,PredKills
0,Icebox,TL,Keiko,jett,238,22,15,1,3,False,11‑13,2025-05-11,0,24,NAVI,14.693905
1,Icebox,TL,kamo,gekko,233,18,19,5,5,False,11‑13,2025-05-11,0,24,NAVI,14.327814
2,Icebox,TL,nAts,viper,153,13,16,4,3,False,11‑13,2025-05-11,0,24,NAVI,15.439730
3,Icebox,TL,Serial,killjoy,141,11,16,5,1,False,11‑13,2025-05-11,0,24,NAVI,11.913463
4,Icebox,TL,paTiTek,kayo,133,8,18,12,1,False,11‑13,2025-05-11,0,24,NAVI,12.066979
5,Icebox,NAVI,hiro,killjoy,298,26,14,2,0,True,11‑13,2025-05-11,0,24,TL,13.375859
6,Icebox,NAVI,Ruxic,viper,199,16,15,4,4,True,11‑13,2025-05-11,0,24,TL,13.433296
7,Icebox,NAVI,koalanoob,omen,195,15,16,13,5,True,11‑13,2025-05-11,0,24,TL,14.642821
8,Icebox,NAVI,ANGE1,tejo,130,13,13,3,2,True,11‑13,2025-05-11,0,24,TL,13.519594
9,Icebox,NAVI,Shao,gekko,160,14,14,4,0,True,11‑13,2025-05-11,0,24,TL,13.798928


In [72]:
est = predict_kills("Xeppa", "Lotus", "MIBR", 28)
print(f"Predicted kills: {est:.1f}")

Predicted kills: 15.0
